# Alpha Optimization in a Gauss-Markov Mobility Model


In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import pandas as pd
import scipy
import numpy as np
from radp_library import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
params = {
    "ue_tracks_generation": {
            "params": {
                "simulation_duration": 3600,
                "simulation_time_interval_seconds": 0.01,
                "num_ticks": 50,
                "num_batches": 1,
                "ue_class_distribution": {
                    "stationary": {
                        "count": 10,
                        "velocity": 0,
                        "velocity_variance": 1
                    },
                    "pedestrian": {
                        "count": 5,
                        "velocity": 2,
                        "velocity_variance": 1
                    },
                    "cyclist": {
                        "count": 5,
                        "velocity": 5,
                        "velocity_variance": 1
                    },
                    "car": {
                        "count": 12,
                        "velocity": 20,
                        "velocity_variance": 1
                    }
                },
                "lat_lon_boundaries": {
                    "min_lat": -90,
                    "max_lat": 90,
                    "min_lon": -180,
                    "max_lon": 180
                },
                "gauss_markov_params": {
                    "alpha": 0.5,
                    "variance": 0.8,
                    "rng_seed": 42,
                    "lon_x_dims": 100,
                    "lon_y_dims": 100,
                    "// TODO": "Account for supporting the user choosing the anchor_loc and cov_around_anchor.",
                    "// Current implementation": "the UE Tracks generator will not be using these values.",
                    "// anchor_loc": {},
                    "// cov_around_anchor": {}
            }
        }
    }
}

## Generate Data Set 1

In [ ]:
data1 = get_ue_data(params)

In [ ]:
data1.head(10)

## Plot Dataset 1 

In [ ]:
plot_ue_tracks(data1)

In [ ]:
velocity = preprocess_ue_data(data1)

In [ ]:
velocity

## Alpha Initialization

In [ ]:
alpha0 = np.random.choice(np.arange(0, 1.1, 0.1))

In [ ]:
print("Alpha0:",alpha0)

## Regress to Find Alpha 1

In [ ]:
alpha1 = get_predicted_alpha(data1,alpha0)

In [ ]:
alpha1

## Generating new data using alpha 1


In [ ]:
#Changing alpha value to alpha1 in the params dictionary
params['ue_tracks_generation']['params']['gauss_markov_params']['alpha'] = alpha1

In [ ]:
print(params['ue_tracks_generation']['params']['gauss_markov_params']['alpha'])

In [ ]:
data2 = get_ue_data(params)

In [ ]:
data2

## Plot Dataset 2

In [ ]:
plot_ue_tracks(data2)

## Regress to Find Alpha 2

In [ ]:
alpha2 =  get_predicted_alpha(data1,alpha1)

In [ ]:
alpha2

## Comparison Plot of Dataset 1 and Dataset2

In [ ]:
plot_ue_tracks_side_by_side(data1, data2)